In [1]:
from datetime import datetime


class Sensor:

    def __init__(self, p_sensor_json):
        self._json = p_sensor_json
        self._last_update = datetime.now()

    def update(self, p_sensor):
        self._last_update = datetime.now()
        self._json['status'] = p_sensor.status

    def check_status(self, p_timeout):
        i_timedelta = (datetime.now() - self._last_update)
        i_since_last = (i_timedelta.microseconds + (i_timedelta.seconds + i_timedelta.days * 24 * 3600) * 10 ** 6) / 10 ** 6
        if i_since_last > p_timeout:
            self._json['status'] = 'offline'

    @property
    def id(self):
        return self._json.get('id')

    @property
    def type(self):
        return self._json.get('type')

    @property
    def status(self):
        return self._json.get('status', 'offline')

    @property
    def address(self):
        return self._json.get('address', str(('localhost', -1)))

    @address.setter
    def address(self, p_address):
        self._json['address'] = str(p_address)

        
class ActiveSensor(Sensor):

    def __init__(self, p_sensor_json):
        super().__init__(p_sensor_json)

    def __del__(self):
        pass
    
    
class Config(ActiveSensor):

    def __init__(self, p_file_name):
        p_file_name = os.path.normpath(p_file_name)
        if os.path.exists(p_file_name):
            with open(p_file_name) as f:
                self._json = yaml.load(f)
        else:
            self._json = dict()
        super().__init__(p_file_name)

class Router(Config):

    def __init__(self, p_router_json):
        super().__init__(p_router_json)

    @property
    def type(self):
        return 'router'

class Kettle(Config):

    def __init__(self, p_kettle_json):
        super().__init__(p_kettle_json)

    @property
    def type(self):
        return 'kettle'

я бы вложил классы роутер и тп в кофиг, который запускает скачивание и присваевает скачанным данным атрибуты класса эктив сенсор, который присвает скачанным данным атрибуты класса sensor

то есть мы сможем вызвывать методы адресс и тд и тп все что прописаны в сенсор

далее по фридж - где прописаны active и passive sensor emulator?? Их надо добавить и то же куда-то вложить. скорее всего в сенсор. а эктив сенсор симулатор заменит класс эктив сенсор, а пэссив сенсор добавится.

In [ ]:
class Fridge(ActiveSensorEmulator) # cкорее всего на конфиг надо будет заменить:

    def __init__(self, p_fridge_json):
        super().__init__(p_fridge_json)
        self._json['type'] = 'fridge'

    @property
    def temperature1(self):
        return self._json.get('temperature1')

    @property
    def temperature2(self):
        return self._json.get('temperature2')

    @temperature1.setter
    def kettle_state(self, p_temperature):
        self._json['temperature1'] = p_temperature

    @temperature2.setter
    def temperature(self, p_temperature):
        self._json['temperature2'] = p_temperature

In [ ]:
далее нам надо power supply

In [ ]:
import asyncore
#наверное надо PassiveSensorEmulator вложить в sensor (не в конфиг, тк тут другой тип загрузки, видимо жругие сигналы дает прибор)
#asyncore.dispatcher_with_send вложить в PassiveSensorEmulator
#powerSupply в PowerSupplyHandler
# соответсвенно везде super прописать
 

class PowerSupplyHandler(asyncore.dispatcher_with_send):

    def __init__(self, p_sock):
        super().__init__(p_sock)

    def handle_read(self):
        data = self.recv(8192)
        if data:
            i_json = json.loads(data.decode('utf-8'))
            print(str(self.addr) + ': ' + str(i_json))

    def handle_connect(self):
        pass

    def handle_close(self):
        self.close()

    def handle_write(self):
        pass


class PowerSupply(PassiveSensorEmulator):

    def __init__(self, p_power_supply_json, host, port):
        super().__init__(p_power_supply_json, host, port, PowerSupplyHandler)

далее должно идти монитор.py а монитор я бы сделал родителем! сенсора! то есть самым первым классом в цепочке
и оставила бы там ---см.ниже + прописала бы что-то или изменила на что-то register и examine


In [ ]:
 def register_sensors(self):
        pass

    def examine_sensors(self):
        pass


    def print_sensors(self):
        print("-{sensor:-<20s}-{type:-<20s}-{address:-<20s}-{status:-<20s}-".format(sensor='', type='', address='', status=''))
        print("|{sensor:^20s}|{type:^20s}|{address:^20s}|{status:^20s}|".format(sensor='ID', type='TYPE', address='ADDRESS', status='STATUS'))
        print("-{sensor:-<20s}-{type:-<20s}-{address:-<20s}-{status:-<20s}-".format(sensor='', type='', address='', status=''))
        for i_id, i_sensor in MessageHandler._sensors.items():
            print("|{sensor:<20s}|{type:<20s}|{address:<20s}|{status:<20s}|".format(sensor=i_id, type=i_sensor.type, address=i_sensor.address, status=i_sensor.status))
        print("-{sensor:-<20s}-{type:-<20s}-{address:-<20s}-{status:-<20s}-".format(sensor='', type='', address='', status=''))

    def run(self):
        self._server.run_threaded()
        try:
            while True:
                # register sensor
                self.register_sensors()
                # examine all sensors
                self.examine_sensors()
                # check statuses
                self.check_statuses()
                os.system('cls' if os.name == 'nt' else 'clear')
                print('Running...')
                # print sensor info
                self.print_sensors()
                time.sleep(1)
        except KeyboardInterrupt:
            pass
        finally:
            pass

    def process_request(self, p_data):
        print('request processing')
        print(p_data)

    @staticmethod
    def main(args=None):
        Monitor(args.config).run()
        return 0